In [41]:
import json
json_args_f = open('args.json')
json_args = json.load(json_args_f)

In [44]:
json_args

{'raw_dir': '../raw',
 'thermo_dlls': '../libs',
 'scan_filter_regex_list': ['ITMS', 'kazoo'],
 'num_workers': 12,
 'parquet_out': './parquet_out'}

In [46]:
!python3 raw_to_parquet.py args.json

Raw Files Directory: ../raw
Thermo dlls Directory:  ../libs
Scan Filter Regex List:  ['ITMS', 'kazoo']
Number of Workers:  12
Parquet File Output Folder:  ./parquet_out
Processing raw file:  MA4383_FFPE_HPVneg_16_071522
Reading Scans...
Processing raw file:  MA4407_06_Mut_082722
Reading Scans...
Processing raw file:  MA4402_01_Mut_082722
Reading Scans...
Processing raw file:  MA4358_FFPE_HPVpos_01_071522
Reading Scans...
Processing raw file:  MA4365_FFPE_HPVpos_08_071522
Reading Scans...
100%|███████████████████████████████████| 66236/66236 [00:12<00:00, 5342.24it/s]
Writing scans to parquet for  MA4402_01_Mut_082722  ...
100%|███████████████████████████████████| 66437/66437 [00:13<00:00, 5014.48it/s]
Writing scans to parquet for  MA4407_06_Mut_082722  ...
100%|███████████████████████████████████| 66354/66354 [00:13<00:00, 5004.23it/s]
Writing scans to parquet for  MA4358_FFPE_HPVpos_01_071522  ...
100%|███████████████████████████████████| 65474/65474 [00:13<00:00, 4929.08it/s]
Writing

In [38]:
!python3 raw_to_parquet.py ../raw -d ../libs -n 12 -sf ITMS kazoo -o ./parquet_out

Raw Files Directory: ../raw
Thermo dlls Directory:  ../libs
Scan Filter Regex List:  ['ITMS', 'kazoo']
Number of Workers:  12
Parquet File Output Folder:  12
Processing raw file:  MA4383_FFPE_HPVneg_16_071522
Reading Scans...
Processing raw file:  MA4407_06_Mut_082722
Reading Scans...
Processing raw file:  MA4402_01_Mut_082722
Reading Scans...
Processing raw file:  MA4358_FFPE_HPVpos_01_071522
Reading Scans...
Processing raw file:  MA4365_FFPE_HPVpos_08_071522
Reading Scans...
100%|███████████████████████████████████| 66236/66236 [00:12<00:00, 5220.07it/s]
Writing scans to parquet for  MA4402_01_Mut_082722  ...
100%|███████████████████████████████████| 65474/65474 [00:13<00:00, 4982.21it/s]
Writing scans to parquet for  MA4383_FFPE_HPVneg_16_071522  ...
100%|███████████████████████████████████| 66437/66437 [00:13<00:00, 5041.14it/s]
Writing scans to parquet for  MA4407_06_Mut_082722  ...
100%|███████████████████████████████████| 66133/66133 [00:13<00:00, 4947.07it/s]
Writing scans to p

In [29]:
!python3 raw_to_parquet.py ../raw -d ../libs -n 3

Raw Files Directory: ../raw
Thermo dlls Directory:  ../libs
Scan Filter Regex List:  ['ITMS']
Number of Workers:  3
Processing raw file:  MA4383_FFPE_HPVneg_16_071522
Reading Scans...
Processing raw file:  MA4407_06_Mut_082722
Reading Scans...
Processing raw file:  MA4402_01_Mut_082722
Reading Scans...
100%|███████████████████████████████████| 66236/66236 [00:11<00:00, 5534.95it/s]
./MA4402_01_Mut_082722.parquet
Writing scans to parquet for  MA4402_01_Mut_082722  ...
100%|███████████████████████████████████| 66437/66437 [00:12<00:00, 5420.87it/s]
./MA4407_06_Mut_082722.parquet
Writing scans to parquet for  MA4407_06_Mut_082722  ...
100%|███████████████████████████████████| 65474/65474 [00:12<00:00, 5219.50it/s]
./MA4383_FFPE_HPVneg_16_071522.parquet
Writing scans to parquet for  MA4383_FFPE_HPVneg_16_071522  ...
Writing raw file  MA4402_01_Mut_082722  took  31.574242115020752  seconds
Processing raw file:  MA4358_FFPE_HPVpos_01_071522
Reading Scans...
 40%|█████████████▉               